# Load data

In [4]:
#imports
import numpy as np
import csv

import tensorflow as tf
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score


In [3]:

dataset = np.load("training-dataset.npz")
x_data = dataset['x']
y_data = dataset['y']
task2_data = np.load("test-dataset.npy")


# Preprocessing


In [5]:
def reshape_28(array):
    new = []
    for i, x in enumerate(array):
        
        new.append(np.reshape(x,(28,28)))
    return np.array(new)

In [6]:
def cropped_image(image, filter_size):
    from numpy import mean
    image_size = np.shape(image)[0]
    cropped_image = []
# Vertical movement of the filter
    for v in np.arange(0,image_size, filter_size):

# Horizontal movement of the filter
        for x in np.arange(0,image_size, filter_size):
            cropped_image.append(mean(image[v:v+filter_size,x: x + filter_size]))
    return cropped_image

def width(image):
    width_list = []
    for i, x in enumerate(np.arange(0,np.shape(image)[0] -1)):
        if sum(image[:,x]) > 300:
            width_list.append(i)
            break
    for i, x in enumerate(np.arange(np.shape(image)[0] -1, 0, -1)):
        if sum(image[:,x]) > 300:
            width_list.append(np.shape(image)[0] - i)
            break
    if len(width_list) != 2:
        return 20
    return width_list[1] - width_list[0]    

def heigth(image):
    heigth_list = []
    for i, x in enumerate(np.arange(0,np.shape(image)[0] -1)):
        if sum(image[x,:]) > 300:
            heigth_list.append(i)
            break
    for i, x in enumerate(np.arange(np.shape(image)[0] -1, 0, -1)):
        if sum(image[x,:]) > 300:
            heigth_list.append(np.shape(image)[0] - i)
            break
    if len(heigth_list) != 2:
        return 20
    return heigth_list[1] - heigth_list[0]    

def density(image):
    return  np.sum(image) / (255 * 28 *28)

def edge_count(image):
    flat = image.flatten()
    safety_margin = int(0.05*255)
    edge_count = 0
    for x in flat:
        if x > 0 + safety_margin and x < 255 - safety_margin:
            edge_count += 1
    return edge_count

# Feature engineering task1

In [7]:
matrix_28 = reshape_28(x_data)
new_features = []
for x in matrix_28:
    new_features.append(cropped_image(x,2))

#two for loops.. could be one
for i, x in enumerate(new_features):
    new_features[i] = np.append(x, cropped_image(matrix_28[i],4))
    new_features[i] = np.append(new_features[i], cropped_image(matrix_28[i],7))
    new_features[i] = np.append(new_features[i], cropped_image(matrix_28[i],14))    
    new_features[i] = np.append(new_features[i],width(matrix_28[i]))
    new_features[i] = np.append(new_features[i],heigth(matrix_28[i]))
    new_features[i] = np.append(new_features[i],density(matrix_28[i]))
    new_features[i] = np.append(new_features[i],edge_count(matrix_28[i]))



# Model 1 (NN)

In [8]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
x_data = scaler.fit_transform(new_features)

In [9]:
#Split for model NN
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.9, random_state=999)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.7777778, random_state=999)

In [10]:
from sklearn.preprocessing import LabelBinarizer
onehot = LabelBinarizer()
y_train = onehot.fit_transform(y_train)
y_val   = onehot.transform(y_val)

# Create NN

In [11]:
# Hyperparameters:
hidden_nodes = 1024 #?
output_size = 26
layer = 'selu'
kernel_init = 'lecun_normal'
NUM_EPOCHS = 100 
eta = 0.00005
regularizer = tf.keras.regularizers.l2(0.01)
nadam = tf.keras.optimizers.Nadam(learning_rate=eta, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam")
adam = tf.keras.optimizers.Adam(
    learning_rate= eta,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
)

model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1024, input_dim=269, activation=layer, kernel_initializer = kernel_init, use_bias = False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(512, activation=layer, kernel_initializer = kernel_init, use_bias = False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(512, activation=layer, kernel_initializer = kernel_init, use_bias = False), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(512, activation=layer, kernel_initializer = kernel_init, use_bias = False),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
])
model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics = ['accuracy'])

In [12]:
model.fit(X_train,y_train, epochs=1, validation_data = (X_val, y_val), verbose=1)

2730/2730 [==============================] - 79s 28ms/step - loss: 1.7781 - accuracy: 0.4893 - val_loss: 0.7879 - val_accuracy: 0.7635


In [13]:
#print accuracy 
y_pred = model.predict(X_test)

y_pred = onehot.inverse_transform(y_pred)
acc = accuracy_score(y_test, y_pred)
print("Accuracy " + str(acc))


Accuracy 0.7569711538461539


# Model 2 (Extremely randomized tree)

In [14]:
#Split for model forest tree without scaling
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_features, y_data, train_size=0.9, random_state=999)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.7777778, random_state=999)

In [15]:
from sklearn.ensemble import ExtraTreesClassifier 
random_forest = ExtraTreesClassifier(n_estimators = 1000, random_state=999, verbose = 2, criterion = 'entropy')
random_forest.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 1000


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building tree 41 of 1000
building

building tree 322 of 1000
building tree 323 of 1000
building tree 324 of 1000
building tree 325 of 1000
building tree 326 of 1000
building tree 327 of 1000
building tree 328 of 1000
building tree 329 of 1000
building tree 330 of 1000
building tree 331 of 1000
building tree 332 of 1000
building tree 333 of 1000
building tree 334 of 1000
building tree 335 of 1000
building tree 336 of 1000
building tree 337 of 1000
building tree 338 of 1000
building tree 339 of 1000
building tree 340 of 1000
building tree 341 of 1000
building tree 342 of 1000
building tree 343 of 1000
building tree 344 of 1000
building tree 345 of 1000
building tree 346 of 1000
building tree 347 of 1000
building tree 348 of 1000
building tree 349 of 1000
building tree 350 of 1000
building tree 351 of 1000
building tree 352 of 1000
building tree 353 of 1000
building tree 354 of 1000
building tree 355 of 1000
building tree 356 of 1000
building tree 357 of 1000
building tree 358 of 1000
building tree 359 of 1000
building tre

building tree 638 of 1000
building tree 639 of 1000
building tree 640 of 1000
building tree 641 of 1000
building tree 642 of 1000
building tree 643 of 1000
building tree 644 of 1000
building tree 645 of 1000
building tree 646 of 1000
building tree 647 of 1000
building tree 648 of 1000
building tree 649 of 1000
building tree 650 of 1000
building tree 651 of 1000
building tree 652 of 1000
building tree 653 of 1000
building tree 654 of 1000
building tree 655 of 1000
building tree 656 of 1000
building tree 657 of 1000
building tree 658 of 1000
building tree 659 of 1000
building tree 660 of 1000
building tree 661 of 1000
building tree 662 of 1000
building tree 663 of 1000
building tree 664 of 1000
building tree 665 of 1000
building tree 666 of 1000
building tree 667 of 1000
building tree 668 of 1000
building tree 669 of 1000
building tree 670 of 1000
building tree 671 of 1000
building tree 672 of 1000
building tree 673 of 1000
building tree 674 of 1000
building tree 675 of 1000
building tre

building tree 954 of 1000
building tree 955 of 1000
building tree 956 of 1000
building tree 957 of 1000
building tree 958 of 1000
building tree 959 of 1000
building tree 960 of 1000
building tree 961 of 1000
building tree 962 of 1000
building tree 963 of 1000
building tree 964 of 1000
building tree 965 of 1000
building tree 966 of 1000
building tree 967 of 1000
building tree 968 of 1000
building tree 969 of 1000
building tree 970 of 1000
building tree 971 of 1000
building tree 972 of 1000
building tree 973 of 1000
building tree 974 of 1000
building tree 975 of 1000
building tree 976 of 1000
building tree 977 of 1000
building tree 978 of 1000
building tree 979 of 1000
building tree 980 of 1000
building tree 981 of 1000
building tree 982 of 1000
building tree 983 of 1000
building tree 984 of 1000
building tree 985 of 1000
building tree 986 of 1000
building tree 987 of 1000
building tree 988 of 1000
building tree 989 of 1000
building tree 990 of 1000
building tree 991 of 1000
building tre

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed: 10.9min finished


ExtraTreesClassifier(criterion='entropy', n_estimators=1000, random_state=999,
                     verbose=2)

In [16]:
from sklearn.metrics import accuracy_score

y_pred = random_forest.predict(X_test)
random_forest_acc = accuracy_score(y_test, y_pred)
print("Accuracy " + str(random_forest_acc))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Accuracy 0.887099358974359


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   10.4s finished


# Preprocessing task 2

In [17]:
def clean_borders(dirty):
    image = dirty.copy()
    image[0] = np.zeros(len(image[0]))
    image[-1] = np.zeros(len(image[0]))
    image[:,0] = np.zeros(len(image[:,0]))
    image[:,-1] = np.zeros(len(image[:,0]))
    return image

def filter1x1(dirty):
    image = dirty.copy()
    for v in np.arange(1,29):
        for h in np.arange(1,139):
            PXL = 0
            PXL += (sum(image[v-1,h-1:h+2]) + sum(image[v+1,h-1:h+2]) + \
            sum(image[v-1:v+2,h-1]) + sum(image[v-1:v+2,h+1] ))
            if PXL < 255 and image[v,h] != 0:
                image[v,h] = 0
    return image

def filter2x2(dirty):
    image = dirty.copy()    
    for v in np.arange(1,28):
        for h in np.arange(1,138):
            PXL = 0
            PXL += (sum(image[v-1,h-1:h+3]) + sum(image[v+2,h-1:h+3]) + \
            sum(image[v-1:v+3,h-1]) + sum(image[v-1:v+3,h+2]) )
            if PXL < 255 and np.sum(image[v:v+2,h:h+2]) != 0:
                image[v:v+2,h:h+2] = np.zeros(4).reshape((2,2))

    return image

def filter3x3(dirty):
    image = dirty.copy()
    for v in np.arange(1,27):
        for h in np.arange(1,137):
            PXL = 0
            PXL += (sum(image[v-1,h-1:h+4]) + sum(image[v+3,h-1:h+4]) + \
            sum(image[v-1:v+4,h-1]) + sum(image[v-1:v+4, h+3]))
            if PXL < 360 and np.sum(image[v:v+3,h:h+3]) > 255 and np.sum(image[v:v+3,h:h+3]) < 1720:
                image[v:v+3,h:h+3] = np.zeros(9).reshape((3,3))
    return image

def filter3x1(dirty):
    image = dirty.copy()    
    for v in np.arange(1,27):
        for h in np.arange(1,139):
            PXL = 0
            PXL += (sum(image[v-1:v+4,h-1]) + sum(image[v-1:v+4,h+1]) +\
            sum(image[v+3,h-1:h+2]) + sum(image[v-1,h-1:h+2]) )
            if PXL < 255 and np.sum(image[v:v+3,h]) != 0:
                image[v:v+3,h] = np.zeros(3).reshape((1,3))
    return image

def filter1x3(dirty):
    image = dirty.copy()
    for v in np.arange(1,29):
        for h in np.arange(1,137):
            PXL = 0
            PXL += (sum(image[v-1,h-1:h+4]) + sum(image[v+1,h-1:h+4]) + \
            sum(image[v-1:v+2,h-1]) + sum(image[v-1:v+2,h+3]))
            if PXL < 255 and np.sum(image[v,h:h+3]) != 0:
                image[v,h:h+3] = np.zeros(3).reshape((1,3))
    return image

def negative_clean(dirty):
    image = dirty.copy()
    PXL = 100
    for v in np.arange(1,29):
        for h in np.arange(1,139):
            if sum(image[v-1,h-1:h+2]) > PXL and sum(image[v+1,h-1:h+2]) > PXL and \
            sum(image[v-1:v+2,h-1]) > PXL and sum(image[v-1:v+2,h+1]) > PXL and \
            image[v,h] < 30 and np.sum(image[v-1:v+2,h-1:h+2]) > 1000:
                image[v,h] = (np.sum(image[v-1:v+2,h-1:h+2]) + 200) / 9
    return image

def clean_those_mf_pixels(image):
    clean = clean_borders(image)
    return negative_clean(filter1x1(filter1x3(filter3x1(filter2x2(filter3x3(clean))))))

In [18]:
def vertical_separator(image):
    letter_list = []
    identifier = []
    i = 0
    
    # While-loop that detects start and beginning of individual letter in a 'Captcha'.
    # A 28x28 frame around the detected center of the letter is stored in the letter_list.
    while i < (np.shape(image)[1] -1):
        if np.sum(image[:,i:i+2]) > 640:
            identifier.append(i)
        
        elif sum(identifier) != 0:
            CTR = np.int(np.mean(identifier))
            if CTR < 12:
                CTR = 12
            elif CTR > 124:
                CTR = 124
            x = image[1:-1, CTR-12: CTR+16]
            letter_list.append(x)
            identifier = []
            i+= 5

        
        i += 1
            
# While the above function is prone to error, the following if statements check whether the desired results are present. Otherwise different separator is used.

    
    if len(letter_list) == 4:
        return letter_list
    
    if len(letter_list) == 3:
        return letter_list_3(image)
    
    if len(letter_list) == 2:
        return letter_list_2(image)

    if len(letter_list) == 1:
        return letter_list_1(image)
    
    if len(letter_list) > 4:
        return letter_list_plus(letter_list)
    
    pass

def letter_list_3(image):

    letter_list = []
    identifier = []
    i = 0
        
    while i < (np.shape(image)[1] -1):
        if np.sum(image[:,i:i+2]) > 640:
            identifier.append(i)
# Until now the separator works the same as in the parent function. The following part is different.
# The frame that is being recorded is bigger (48x48) if possible, so that combined letters can be separated later on.
        elif sum(identifier) != 0:
            CTR = np.int(np.mean(identifier))
            if CTR < 12:
                x = image[1:-1, 0: 28]
            elif CTR < 20:
                x = image[1:-1, CTR-12: CTR+16]
            elif CTR > 112:
                CTR = 112
                x = image[1:-1, CTR-20: CTR+28]
            else:
                x = image[1:-1, CTR-20: CTR+28]
            letter_list.append(x)
            identifier = []
            i+= 5

        i += 1

    density = []
# The identified letters are being transformed into a single value, stored in the density list.
# Value is based on the sum of all pixels in the frame.
# The letter with the highest density is split into two 28x28 frames, this is often the double letter.
    for x in letter_list:
        density = np.append(density, np.sum(x))
        double_letter = letter_list[np.argmax(density)]
        if np.shape(double_letter) == (28,28):
            try:
                double_letter = letter_list[np.argmax(density) + 1]
            except:
                double_letter = letter_list[np.argmax(density) - 1]
    single_letter1 = double_letter[:, 0:28]
    single_letter2 = double_letter[:,20:]
    

# remaining 48x48 frames are being transformed into 28x28 frames.    
    for i, x in enumerate(letter_list):
        if np.shape(x) != (28,28):
            letter_list[i] = x[:,10:38]

# The separated letters are placed back on the correct position and the old combined frame is removed.
    good_list = []
    for i, x in enumerate(letter_list):
        if i == np.argmax(density):
            good_list.append(single_letter1)
            good_list.append(single_letter2)
        else:
            good_list.append(x)
        
    return good_list
        
def letter_list_2(image):

    letter_list = []
    identifier = []
    i = 0
        
    while i < (np.shape(image)[1] -1):
        if np.sum(image[:,i:i+2]) > 640:
            identifier.append(i)
# Until now the separator works the same as in the parent function. The following part is different.
# The frame that is being recorded is bigger (48x48) if possible, so that combined letters can be separated later on.
        elif sum(identifier) != 0:
            CTR = np.int(np.mean(identifier))
            if CTR < 20:
                CTR = 20
            if CTR > 112:
                CTR = 112
            x = image[1:-1, CTR-20: CTR+28]
            letter_list.append(x)
            identifier = []
            i+= 5

        i += 1

    new_list = []
    for x in letter_list:
        new_list.append(x[:, 0:28])
        new_list.append(x[:,20:])
    return new_list

def letter_list_1(image):
    letter_list = []
    identifier = []
    i = 0
    
# While-loop that detects start and beginning of individual letter in a 'Captcha'.
# A 28x28 frame around the detected center of the letter is stored in the letter_list.
    while i < (np.shape(image)[1] -1):
        if np.sum(image[:,i:i+2]) > 2295:
            identifier.append(i)
        
        elif sum(identifier) != 0:
            CTR = np.int(np.mean(identifier))
            if CTR < 12:
                CTR = 12
            elif CTR > 124:
                CTR = 124
            x = image[1:-1, CTR-12: CTR+16]
            letter_list.append(x)
            identifier = []
            i+= 5

        
        i += 1
    
    return letter_list

def letter_list_plus(letter_list):

    for x in np.arange(len(letter_list) - 4):
        density = []
        for x in letter_list:
            density = np.append(density, np.sum(x))

        for i, x in enumerate(letter_list):
            if i == np.argmin(density):
                continue
            letter_list[i] = x
    return letter_list[:4]

In [24]:
clean = []

for x in task2_data:
        clean.append(clean_those_mf_pixels(x))
        
task2_clean_separated = []
for x in clean:
    task2_clean_separated.append(vertical_separator(x))
    
task2_cleaned = task2_clean_separated.copy()

In [20]:


def task2_features(x):
    features_list = []
    features_list = np.append(features_list,cropped_image(x,2))
    features_list = np.append(features_list, cropped_image(x,4))
    features_list = np.append(features_list, cropped_image(x,7))
    features_list = np.append(features_list, cropped_image(x,14))    
    features_list = np.append(features_list,width(x))
    features_list = np.append(features_list,heigth(x))
    features_list = np.append(features_list,density(x))
    features_list = np.append(features_list,edge_count(x))
    return np.array(features_list)

In [25]:
task2_data_preprocessed = []
for n in task2_cleaned:
    per_4 = []
    for x in n:
        per_4.append(task2_features(x))
    task2_data_preprocessed.append(per_4)
    

# end task 2

# Create feature data

In [22]:
import numpy as np
import csv 


#Get the top scores from the predictions and replace the lowest score 
#from all letters with the next best performing predictions
def getTopScores(all_letter_scores, all_letters):
    secondUsedIndex = []
    thirdUsedIndex = []
    fourthUsedIndex = []
    
    resultList = []
    firstScores = []
    secondScores = []
    thirdScores = []
    fourthScores = []
    fifthScores = []
    
    firstPredictions = []
    secondPredictions = []
    thirdPredictions = []
    fourthPredictions = []
    fifthPredictions = []
    
    index = 0
    #add all predictions to a list based on their score
    for scores in all_letter_scores: 
        firstScores.append(scores[0]) 
        secondScores.append(scores[1])
        thirdScores.append(scores[2])
        fourthScores.append(scores[3])
        fifthScores.append(scores[4])
        
        firstPredictions.append(all_letters[index][0]) 
        secondPredictions.append(all_letters[index][1])
        thirdPredictions.append(all_letters[index][2])
        fourthPredictions.append(all_letters[index][3])
        fifthPredictions.append(all_letters[index][4])
        
        index +=1    
    scoreList = [[]]*5
    #update the score_list with the highest probability for the 4 letters
    for i in range(5):   
        
        completeStr = ""    
        indexLowest = firstScores.index(min(firstScores))
        for strScore in firstPredictions:
            #for the sake of reference add to new object
            completeStr += str(strScore).zfill(2)
        scoreList[i] = completeStr   
        
        #if the index is not used before get value from second position otherwise from next
        if (indexLowest not in secondUsedIndex):
            secondUsedIndex.append(indexLowest)
            firstScores[indexLowest] = secondScores[indexLowest]
            firstPredictions[indexLowest] = secondPredictions[indexLowest]
        elif (indexLowest not in thirdUsedIndex):
            thirdUsedIndex.append(indexLowest)
            firstScores[indexLowest] = thirdScores[indexLowest]
            firstPredictions[indexLowest] = thirdPredictions[indexLowest]
        elif (indexLowest not in fourthUsedIndex):
            fourthUsedIndex.append(indexLowest)
            firstScores[indexLowest] = fourthScores[indexLowest]
            firstPredictions[indexLowest] = fourthPredictions[indexLowest]
        else: 
            firstScores[indexLowest] = fifthScores[indexLowest]
            firstPredictions[indexLowest] = fifthPredictions[indexLowest]
    
    return scoreList 

def predictImages(images, model):
    output_list = []
    output_list.append(model.predict_proba(images))

    all_letter_scores = []
    all_letters = []
    letterIndex = range(1, 27)
    #loop over the predictions
    for pred in output_list[0]:
       
        letterScoreList = []
        letterList = []
        
        #get the top 5 predictions
       
        score = sorted(zip(pred, letterIndex), reverse=True)[:5]
        letterScoreList.append(score[0][0])
        letterList.append(score[0][1])

        letterScoreList.append(score[1][0])
        letterList.append(score[1][1])

        letterScoreList.append(score[2][0])
        letterList.append(score[2][1])

        letterScoreList.append(score[3][0])
        letterList.append(score[3][1])

        letterScoreList.append(score[4][0])
        letterList.append(score[4][1])

        all_letter_scores.append(letterScoreList)
        all_letters.append(letterList)
    #get the top 5 digit predictions
    return getTopScores(all_letter_scores, all_letters)

#write prediction to file
def toPredictionFile(all_scores):
    with open('prediction.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(all_scores)

In [26]:
all_scores = []
for features in task2_data_preprocessed:
    all_scores.append(predictImages(features, random_forest))

#export to file
toPredictionFile(all_scores)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurr

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finis

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finis

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finis

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finis

...

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finis

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finis

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finis

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finis

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finis